First we started by reading the csv file 'captions.txt' and then displaying the first five rows

In [37]:

from os import truncate

import pandas as pd
from keras import Layer
from tensorflow.python.keras import Sequential

captions_path = "../data/captions/captions.txt"

df = pd.read_csv(captions_path)
print(df.head())


                       image  \
0  1000268201_693b08cb0e.jpg   
1  1000268201_693b08cb0e.jpg   
2  1000268201_693b08cb0e.jpg   
3  1000268201_693b08cb0e.jpg   
4  1000268201_693b08cb0e.jpg   

                                             caption  
0  A child in a pink dress is climbing up a set o...  
1              A girl going into a wooden building .  
2   A little girl climbing into a wooden playhouse .  
3  A little girl climbing the stairs to her playh...  
4  A little girl in a pink dress going into a woo...  


Now we want group all rows that have the same image filename together, keeping only the caption column, then for each group we convert all captions into a python list, and finally convert the pandas object into a dictionary

In [38]:
image_to_captions = (
    df.groupby('image')['caption'].apply(list).to_dict()
)

print("Number of images: ", len(image_to_captions))

Number of images:  8091


Creating a new column for captions in lower case and without punctuation

In [39]:

import string

#Removing the upper cases
df['lower_caption'] = df['caption'].str.lower()

#Function to remove the punctuation
def remove_punctuation(lower_caption):
    for char in string.punctuation:
        lower_caption = lower_caption.replace(char, '')
    return lower_caption

df['lower_caption'] = df['lower_caption'].apply(remove_punctuation)

#Comparing the new column with the old column
df.sample(10)



,image,caption,lower_caption
11488,2565685680_c30972455d.jpg,The three black dogs are swimming very close t...,the three black dogs are swimming very close t...
16832,2914737181_0c8e052da8.jpg,A man waterskis while being pulled .,a man waterskis while being pulled
13677,2711075591_f3ee53cfaa.jpg,Two people are scuba diving in blue water with...,two people are scuba diving in blue water with...
34196,3679341667_936769fd0c.jpg,"Two people kick boxing , with others spectating .",two people kick boxing with others spectating
4924,2094543127_46d2f1fedf.jpg,A girl with red hair is sitting on a crocodile...,a girl with red hair is sitting on a crocodile...
12975,2666205903_8d287669e1.jpg,A child is holding a colorful object and overl...,a child is holding a colorful object and overl...
32159,359173181_a75c950aeb.jpg,"The brown , furry animal is walking in the sno...",the brown furry animal is walking in the snow...
98,102351840_323e3de834.jpg,A person standing on a frozen lake .,a person standing on a frozen lake
30851,3541141771_67d305c873.jpg,"Two boys , one is climbing a roof , the other ...",two boys one is climbing a roof the other is...
1512,1332815795_8eea44375e.jpg,A little girl is swinging in a baby swing on t...,a little girl is swinging in a baby swing on t...


Tokenization for each caption

In [40]:

import nltk
from nltk.tokenize import word_tokenize
#Was having trouble with the import word_tokenize so had to add the line below
#nltk.download('punkt_tab')

#Creating a new column for tokens for each caption
df['tokens'] = df['lower_caption'].apply(lambda x: word_tokenize(x))
df.head()






,image,caption,lower_caption,tokens
0,1000268201_693b08cb0e.jpg,A child in a pink dress is climbing up a set o...,a child in a pink dress is climbing up a set o...,"[a, child, in, a, pink, dress, is, climbing, u..."
1,1000268201_693b08cb0e.jpg,A girl going into a wooden building .,a girl going into a wooden building,"[a, girl, going, into, a, wooden, building]"
2,1000268201_693b08cb0e.jpg,A little girl climbing into a wooden playhouse .,a little girl climbing into a wooden playhouse,"[a, little, girl, climbing, into, a, wooden, p..."
3,1000268201_693b08cb0e.jpg,A little girl climbing the stairs to her playh...,a little girl climbing the stairs to her playh...,"[a, little, girl, climbing, the, stairs, to, h..."
4,1000268201_693b08cb0e.jpg,A little girl in a pink dress going into a woo...,a little girl in a pink dress going into a woo...,"[a, little, girl, in, a, pink, dress, going, i..."


Building the vocabulary

In [41]:
from collections import Counter

#Couting all the words
word_counter = Counter()

for i in df['tokens']:
    word_counter.update(i)

#Defining special tokens
#To fill empty positions
PAD_TOKEN = "<PAD>"
#When a word is not in the vocabulary
UNK_TOKEN = "<UNK>"

#Creating a vocabulary
word_to_idx = {
    PAD_TOKEN: 0,
    UNK_TOKEN: 1
}

for word in word_counter:
    #Auto increments the ID generator
    word_to_idx[word] = len(word_to_idx)


print("Vocabulary size:", len(word_to_idx))
print("Sample entries:", list(word_to_idx.items())[:10])


Vocabulary size: 8829
Sample entries: [('<PAD>', 0), ('<UNK>', 1), ('a', 2), ('child', 3), ('in', 4), ('pink', 5), ('dress', 6), ('is', 7), ('climbing', 8), ('up', 9)]


Convert the tokens to integers


In [42]:

#Creating a variable for unk token index
UNK_IDX = word_to_idx[UNK_TOKEN]

sequence = []

#For loop to check if the word is in the tokens dataframe; if it is just adds the index; if it's not it will add the unk index
for tokens in df['tokens']:
    seq = []
    for word in tokens:
        if word in word_to_idx:
            seq.append(word_to_idx[word])
        else:
            seq.append(UNK_IDX)
    #Appending to the empty sequence list
    sequence.append(seq)

print("Original: ", df['tokens'][1])
print("Sequence: ", sequence[1])



Original:  ['a', 'girl', 'going', 'into', 'a', 'wooden', 'building']
Sequence:  [2, 16, 17, 18, 2, 19, 20]


Organizing

In [43]:

#Dropping caption, and lower_caption columns to organize the dataframe since they aren't useful anymore
#Inplace = True to change the original and don't give us just a copy
df.drop(['caption', 'lower_caption'], axis= 'columns', inplace= True)


Sequence padding and truncating


In [44]:

#Checking the biggest sequence and the 95%
import numpy as np
lengths = [len(i) for i in sequence]
print(f"Max: {max(lengths)}, 95th percentile: {np.percentile(lengths, 95)}")

#Truncating sequences > 20
#Padding sequences < 20
from tensorflow.keras.preprocessing.sequence import pad_sequences

max_length = 20

padded_sequences = pad_sequences(
    sequence,
    maxlen = max_length,
    #Padding at the end
    padding = 'post',
    #Truncating at the end
    truncating = 'post'
)

#Printing the first 20 sequences as numpy arrays
print(padded_sequences[:20])
print(padded_sequences.shape)




Max: 36, 95th percentile: 18.0
[[ 2  3  4  2  5  6  7  8  9  2 10 11 12  4 13 14 15  0  0  0]
 [ 2 16 17 18  2 19 20  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 2 21 16  8 18  2 19 22  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 2 21 16  8 23 12 24 25 22  0  0  0  0  0  0  0  0  0  0  0]
 [ 2 21 16  4  2  5  6 17 18  2 19 26  0  0  0  0  0  0  0  0]
 [ 2 27 28 29  2 30 28 31 32  0  0  0  0  0  0  0  0  0  0  0]
 [ 2 27 28 29  2 33 28 34 35 36 37 38 23 39  0  0  0  0  0  0]
 [ 2 27 28 29  2 40 28 35 41 42 31 43 44 36 37  4 23 45  0  0]
 [46 47 11 48 49 50 44 36 37 38 23 39  0  0  0  0  0  0  0  0]
 [46 47 38 51 52 53 36 37  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 2 21 16 54  4 55 56  4 57 11  2 58 59 35 25 60  4  2 61  0]
 [ 2 21 16  7 62  4 57 11  2 63 58 59  0  0  0  0  0  0  0  0]
 [ 2 64 16  4 23 65 66 35 67  4 57 11  2 40 68 35  2 59 38 69]
 [70  7  2 16 35 71 62  4 57 11  2 59 72  0  0  0  0  0  0  0]
 [73 16 35 71 72 74  4 23 65  0  0  0  0  0  0  0  0  0  0  0]
 [ 2 75 76 38  2 77 78 7

Embedding Layer

In [59]:

import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.layers import Embedding, LSTM

#To check the size and length
print(f"Size of the vocabulary: {len(word_to_idx)}")
print(f"Length of each sequence: {padded_sequences.shape}")

vocab_size = len(word_to_idx)
#Default dimension for small / medium datasets (flickr8k)
embedding_dim = 128
input_length = 20

#Building the model
model = models.Sequential([
    Embedding(input_dim = vocab_size, output_dim = embedding_dim, input_length = input_length), LSTM(256) #Standard default
])

model.build(input_shape=(None, input_length))
model.summary()


Size of the vocabulary: 8829
Length of each sequence: (40455, 20)


Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_5 (Embedding)         │ (None, 20, 128)        │     1,130,112 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_4 (LSTM)                   │ (None, 256)            │       394,240 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,524,352 (5.81 MB)

 Trainable params: 1,524,352 (5.81 MB)

 Non-trainable params: 0 (0.00 B)